In [1]:
import findspark
findspark.init('/opt/spark')

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SCARP_Kafka_Test')\
                    .config('spark.jars.packages','org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1')\
                    .config('spark.jars.packages','org.apache.kafka:kafka-clients:2.4.1')\
                    .config("spark.driver.memory", "24g")\
                    .config("spark.executor.memory", "8g")\
                    .getOrCreate()

22/06/23 22:51:17 WARN Utils: Your hostname, GPUServer resolves to a loopback address: 127.0.1.1; using 10.4.10.8 instead (on interface enp2s0)
22/06/23 22:51:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/weitkampe/.ivy2/cache
The jars for the packages stored in: /home/weitkampe/.ivy2/jars
:: loading settings :: url = jar:file:/opt/spark-3.0.1/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-61e4fa83-3e5a-4076-b8fc-0df02c8a41bf;1.0
	confs: [default]
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.3-1 in central
	found org.lz4#lz4-java;1.6.0 in central
	found org.xerial.snappy#snappy-java;1.1.7.3 in central
	found org.slf4j#slf4j-api;1.7.28 in central
:: resolution report :: resolve 116ms :: artifacts dl 3ms
	:: modules in use:
	com.github.luben#zstd-jn

In [3]:
df = spark.readStream.format("kafka")\
    .option("kafka.bootstrap.servers", "localhost:9092")\
    .option("subscribe", "d4p_stream")\
    .option('includeTimestamp', 'true')\
    .load()

In [4]:
data_df = df.select(['value','timestamp'])

In [5]:
query = data_df.writeStream\
        .format('memory')\
        .queryName('d4p_stream')\
        .start()

22/06/23 22:51:20 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f5a696bc-da76-4829-867b-07c485f8db38. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [6]:
spark.sparkContext.setLogLevel("ERROR")

In [14]:
import numpy as np
import ember_modified
from ember_modified.features import PEFeatureExtractor
import time
from IPython.display import clear_output
import pickle
from daal4py import gbt_classification_training, gbt_classification_prediction

In [8]:
GBTmodel = pickle.load(open('./d4p_GBT_model', 'rb'))

In [9]:
predictor = gbt_classification_prediction(2)

In [ ]:
receiving = 100
totalMalware = 0
totalBenign = 0

extractor = PEFeatureExtractor(2)

_df = spark.sql('SELECT * FROM d4p_stream ORDER BY timestamp DESC')
while (_df.count() == 0):
    _df = spark.sql('SELECT * FROM d4p_stream ORDER BY timestamp DESC')

prev_timestamp = _df.select('timestamp').collect()[0].asDict()['timestamp']
        
print("Entering loop")
while (totalMalware+totalBenign < receiving):
    
    # Create time range
    _df = spark.sql('SELECT * FROM d4p_stream WHERE timestamp > \''+str(prev_timestamp)+'\' ORDER BY timestamp DESC')
    
    if _df.count() != 0:

        _df.persist()
        df_data = _df.collect()
        
        prev_timestamp = df_data[0].asDict()['timestamp']
        #print(prev_timestamp)
        
        #start_time = time.time_ns()
        for row in df_data:
    
            raw_data = row.asDict()['value']
        
            features = extractor.feature_vector(bytes(raw_data))
            
            prediction = int(predictor.compute(np.reshape(features, (1,2381)), GBTmodel).prediction.flatten())
            #print(prediction)
            
            if prediction==1:
                totalMalware+=1
            else:
                totalBenign+=1
            
        clear_output(wait=True)
        print("Total Malware: "+str(totalMalware)+"\tTotal Benign: "+str(totalBenign))


Total Malware: 1	Total Benign: 3
